In [1]:
from urllib import request
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd
import json

title_list = []; writer_list = []; score_list = []; genre_list = []; category_list = []; introduction_list = []; age_list = []

In [2]:
def getLists(url):
    html = request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')
    jsondata = json.loads(soup.text)
    
    for i in jsondata['data']:
        t = i.get('title')
        if(t in title_list):
            continue
        else:
            title_list.append(t)
    
            writers = []
            for j in i.get('cartoon').get('artists'):
                writers.append(j.get('name'))
            writer_list.append(writers)
            
            score_list.append(i.get('averageScore'))
            
            age = i.get('ageGrade')
            if age == 19:
                age_list.append(1)
            else:
                age_list.append(0)
    
            genres = []
            for j in i.get('cartoon').get('genres'):
                genres.append(j.get('name'))
            genre_list.append(genres)
    
            categories = []
            for j in i.get('cartoon').get('categories'):
                categories.append(j.get('name'))
            category_list.append(categories)
    
            introduction_list.append(i.get('introduction'))

In [3]:
def getFinishedLists(url):
    html = request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')
    jsondata = json.loads(soup.text)
    
    for i in jsondata['data']:
        t = i.get('title')
        if(t in title_list):
            continue
        else:
            title_list.append(t)
            nickname = i.get('nickname')
    
            writers = []
            for j in i.get('cartoon').get('artists'):
                writers.append(j.get('name'))
            writer_list.append(writers)
            
            score_list.append(i.get('averageScore'))
            
            age = i.get('ageGrade')
            if age == 19:
                age_list.append(1)
            else:
                age_list.append(0)
            
            title_split = t.split('(')[0]
            URL = 'http://webtoon.daum.net/data/pc/search?q=' + parse.quote(title_split.split(' ')[0])
            
            html = request.urlopen(URL).read()
            soup = BeautifulSoup(html,'html.parser')
            jsondata = json.loads(soup.text)
            
            if jsondata['data'].get('webtoon') is None:
                URL = 'http://webtoon.daum.net/data/pc/webtoon/view/' + nickname
                html = request.urlopen(URL).read()
                soup = BeautifulSoup(html,'html.parser')
                jsondata = json.loads(soup.text)
                
                j = jsondata['data'].get('webtoon')
                
                genres = []
                for k in j.get('cartoon').get('genres'):
                    genres.append(k.get('name'))
                genre_list.append(genres)
    
                categories = []
                for k in j.get('cartoon').get('categories'):
                    categories.append(k.get('name'))
                category_list.append(categories)
    
                introduction_list.append(j.get('introduction'))
                
            else:
                t_all = []
                for search_result in jsondata['data'].get('webtoon'):
                    tit = search_result.get('title')
                    t_all.append(tit)
                    if tit == t:
                        break
    
                if t not in t_all:
                    URL = 'http://webtoon.daum.net/data/pc/webtoon/view/' + nickname
                    html = request.urlopen(URL).read()
                    soup = BeautifulSoup(html,'html.parser')
                    jsondata = json.loads(soup.text)
                
                    j = jsondata['data'].get('webtoon')
                
                    genres = []
                    for k in j.get('cartoon').get('genres'):
                        genres.append(k.get('name'))
                    genre_list.append(genres)
    
                    categories = []
                    for k in j.get('cartoon').get('categories'):
                        categories.append(k.get('name'))
                    category_list.append(categories)
    
                    introduction_list.append(j.get('introduction'))
            
                else:
                    j = jsondata['data'].get('webtoon')[t_all.index(t)]
                    
                    genres = []
                    for k in j.get('cartoon').get('genres'):
                        genres.append(k.get('name'))
                    genre_list.append(genres)
    
                    categories = []
                    for k in j.get('cartoon').get('categories'):
                        categories.append(k.get('name'))
                    category_list.append(categories)
    
                    introduction_list.append(j.get('introduction'))

In [4]:
url_mon = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/mon'
url_tue = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/tue'
url_wed = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/wed'
url_thu = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/thu'
url_fri = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/fri'
url_sat = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/sat'
url_sun = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/sun'
url_finished = 'http://webtoon.daum.net/data/pc/webtoon/list_daily_ranking/finished'
url_finished_gen = 'http://webtoon.daum.net/data/pc/webtoon/list_finished/?genre_id='

url_list = [url_mon, url_tue, url_wed, url_thu, url_fri, url_sat, url_sun, url_finished]

for i in url_list:
    getLists(i)
    
getFinishedLists(url_finished_gen)

In [5]:
webtoonfile = pd.DataFrame({'제목': title_list, \
                            '작가': writer_list, \
                            '평점': score_list, \
                            '연령': age_list, \
                            '장르': genre_list, \
                            '카테고리': category_list, \
                            '한줄 스토리': introduction_list})

webtoonfile.to_csv('daum_webtoon_crawling.csv', mode='w')